In [ ]:
import pandas as pd
import psycopg2
#dataset_link="Bike Sharing Original Data.csv" # для чтения с диска
dataset_link=r"https://raw.githubusercontent.com/YBI-Foundation/Dataset/refs/heads/main/Bike%20Sharing%20Original%20Data.csv" # для чтения напрямую с репозитория
df=pd.read_csv(dataset_link, sep=",")

print(f"\nИсходный датафрейм:\n{df}")

# Использование .apply()
# Группировка исходных данных по годам и месяцам
df_groupped=df.groupby(['Year','Month'], sort=False)

def temp_delta(x): # Функция для вычисления разницы между максимальной и минимальной температурой
    return x['Temperature'].max()-x['Temperature'].min()

df_apply=df_groupped.apply(temp_delta, include_groups=False)
print(f"\nРазница между максимальной и минимальной температурой в разбивке по годам и месяцам после применения .apply():\n{df_apply}")

# Использование .transform()
df_transform=df.copy()
def strong_wind_count(x): # Функция для вычисления количества дней со скоростью ветроа, превышающей 15
    c=0
    for i in x:
        if i>15:
            c+=1
    return c
        
df_transform['Strong Wind']=df_groupped[['Wind Speed']].transform(strong_wind_count)
print(f"\nДатафрейм с новым столбцом Strong Wind после применения .transform() к ранее сгруппированным данным:\n{df_transform.head(70)}")
print("Значения в столбце Strong Wind демонстрируют сходство с поведением оконной функции в SQL")

# Использование .map()
df_map=df.copy()
df_map['Temperature F']=df_map['Temperature'].map(lambda t:1.8*t+32) #Лямбда-функция перевода из градусов Цельсия в градусы Фаренгейта
print(f"\nДатафрейм после использования .map() для добавления столбца с температурой по шкале Фаренгейта:\n{df_map}")

# Использование .resample()
# Создание датафрейма из БД dvdrental
query='''
SELECT rental_date, cust.first_name ||' '|| cust.last_name AS customer, staff.first_name ||' '|| staff.last_name AS staff, film.title AS film_title, film.description AS film_description,film.rental_rate AS film_rental_rate  
FROM rental AS rent
LEFT JOIN customer AS cust
ON rent.customer_id=cust.customer_id
LEFT JOIN staff AS staff
ON rent.staff_id=staff.staff_id
LEFT JOIN inventory AS inv
ON rent.inventory_id=inv.inventory_id
LEFT JOIN film AS film
ON inv.film_id=film.film_id'''
sql_conn=psycopg2.connect(dbname='dvdrental', user='postgres', password='postgres', host='localhost', port='5432')
df_dvdrental=pd.read_sql(sql=query, con=sql_conn)

print(f"\nДатафрейм из БД dvdrental:\n{df_dvdrental}")
df_dvdrental_groupped=df_dvdrental.groupby('rental_date').agg({'film_rental_rate':'sum'})
print(f"\nДатафрейм из БД dvdrental после предварительной группировки по датам и агрегирования:\n{df_dvdrental_groupped}")
print(f"\nРезультат после агрегирования по месяцам с использованием .resample():\n{df_dvdrental_groupped.resample('ME').sum()}")
